In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import time
from scipy import signal
import warnings
from sklearn.preprocessing import minmax_scale
import heartpy as hp
warnings.filterwarnings('ignore')

In [2]:
source_dir = 'D:/Code/data/PLETH/'

rawdata_list = os.listdir(source_dir)
print(len(rawdata_list))

6157


In [3]:
Censoring    = 100 * 60 * 25  # 25 min, 데이터에서 앞의 25분과 뒤의 25분을 잘라내기 위한 상수값(hyper parameter)
DataSet      = []
ErrorTracker = []
SampleInfo   = []
RawSampleIDXList = []

path = 'D:/Code/data/'
ParaFolder = 'TestPLETH'

In [4]:
for idx, filename in enumerate(rawdata_list[:2000]):
    print(idx, ': ', filename)
    data = np.load(source_dir + filename, allow_pickle=True)['PLETH'][:,0][Censoring:-Censoring]

    # Nan 값 있는 index 선택,
    ExcCriteria = np.where(np.isnan(data))[0]
    ExcCriteria = np.concatenate([[0], ExcCriteria, [data.shape[0]]])
    IndMaxDif = ExcCriteria[1:] - ExcCriteria[:-1]
    IndMaxDif_enough = np.where(IndMaxDif>100*60*1)[0] # 1 분이상 연속시계열 확보 구간 선정

    if len(IndMaxDif_enough) <= 0: # OBS 확보 안되는 sample은 포함 시키지 않기
        print('Sample excluded due to small obs.')
        # ErrorTracker.append(str(idx)+' excluded due to small obs.')
        print()
        continue

    StartInd = ExcCriteria[IndMaxDif_enough][..., np.newaxis]
    EndInd = ExcCriteria[IndMaxDif_enough+1][..., np.newaxis]
    SelectedPeriod = np.concatenate([StartInd,EndInd], axis=-1)

    SampleIDXVec = np.full((SelectedPeriod.shape[0],1), idx)
    RawSampleIDX = np.concatenate([SampleIDXVec, (SelectedPeriod + Censoring)], axis=1)
    RawSampleIDXList.append(RawSampleIDX)

    for PeriodNum,Period in enumerate(SelectedPeriod):
        print('Sample-'+str(idx)+'-'+str(PeriodNum))
        # SampleInfo.append('Sample-'+str(idx)+'-'+str(PeriodNum))
        DataSet.append(data[Period[0]:Period[1]])
    print()

# np.save(path+ParaFolder+'/SampleInfo',SampleInfo)
# np.save(path+ParaFolder+'/ErrorTracker',ErrorTracker)
np.save(path+ParaFolder+'/DataSet',DataSet)
# np.save(path+ParaFolder+'/RawSampleIDX_raw',RawSampleIDXList)

0 :  1.npz
Sample-0-0

1 :  10.npz
Sample-1-0

2 :  100.npz
Sample-2-0

3 :  1000.npz
Sample-3-0

4 :  1001.npz
Sample-4-0

5 :  1002.npz
Sample-5-0

6 :  1003.npz
Sample-6-0

7 :  1004.npz
Sample-7-0
Sample-7-1

8 :  1005.npz
Sample-8-0
Sample-8-1

9 :  1006.npz
Sample-9-0

10 :  1007.npz
Sample-10-0

11 :  1008.npz
Sample-11-0

12 :  1009.npz
Sample-12-0

13 :  101.npz
Sample-13-0

14 :  1011.npz
Sample-14-0

15 :  1012.npz
Sample-15-0
Sample-15-1

16 :  1013.npz
Sample-16-0
Sample-16-1

17 :  1014.npz
Sample-17-0

18 :  1015.npz
Sample-18-0

19 :  1016.npz
Sample-19-0
Sample-19-1

20 :  1017.npz
Sample-20-0

21 :  1019.npz
Sample-21-0

22 :  102.npz
Sample-22-0
Sample-22-1

23 :  1020.npz
Sample-23-0

24 :  1021.npz
Sample-24-0
Sample-24-1

25 :  1022.npz
Sample-25-0
Sample-25-1

26 :  1023.npz
Sample-26-0

27 :  1024.npz
Sample-27-0
Sample-27-1

28 :  1025.npz
Sample-28-0
Sample-28-1

29 :  1026.npz
Sample-29-0
Sample-29-1
Sample-29-2
Sample-29-3

30 :  1027.npz
Sample-30-0
Sample-

In [5]:
DataSet = np.load(path+ParaFolder+'/DataSet.npy',allow_pickle = True)
len(DataSet)

2823

In [6]:
def get_difference_list(seq):
    seq  = np.concatenate([[0], seq, [0]])
    differ = seq[1:] - seq[:-1]
    differ = differ[1:-1]
    return differ

In [10]:
def check_ppg_normal(ppg, sig_length=2500, fs=100):
    if (np.max(ppg) >= 100) or (np.min(ppg) <= 0):
        return 0

    PPG = minmax_scale(ppg)
    # Transpose the Signals if they are in column format, necessary for peak detection
    if PPG.shape[0] > 1:
        PPG = PPG.T

    try:
        wd_p, _ = hp.process(PPG, sample_rate=fs) # peak 검출
    except Exception:
        return 0

    # Peak
    pks_PPG = wd_p['peaklist'] # peak의 위치 리스트
    pks_PPG = np.array(pks_PPG)
    peak_dist_PPG = get_difference_list(pks_PPG) # peak 간격 리스트
    std_peak_dist_PPG = np.std(peak_dist_PPG) # peak 간격의 std, feature 1
    std_peaks_PPG = np.std(PPG[pks_PPG]) # peak 값의 std, feature 2
    num_peaks_PPG = len(pks_PPG) # peak 갯수, feature 3

    # peak 값들이 고르지 않거나, peak 간격이 고르지 않거나, peak 갯수가 길이에 비해 많거나 적을 경우 => bad signal
    if (std_peaks_PPG > 0.085) or (std_peak_dist_PPG > 10) or (num_peaks_PPG < int(sig_length/(fs*1.5))) or (num_peaks_PPG > int(sig_length/(fs*0.5))):
        return 0

    # 첫번째 peak가 너무 늦게 검출되거나, 마지막 peak가 너무 이르게 검출되면 => bad signal
    if (pks_PPG[0] > 1.5*np.mean(peak_dist_PPG)) or (sig_length - pks_PPG[-1] > 1.5*np.mean(peak_dist_PPG)):
        return 0

    # 추가 feature
    max_min_differ_pks = np.max(PPG[pks_PPG]) - np.min(PPG[pks_PPG]) # 최대 peak와 최소 peak 차이, feature 4
    differ_pks = get_difference_list(PPG[pks_PPG])
    max_differ_pks = max(np.abs(differ_pks)) # peak간 차분값의 최댓값, feature 5

    if (max_min_differ_pks > 0.19) or (max_differ_pks > 0.15):
        return 0

    if np.min(ppg[pks_PPG]) <= 40:
        return 0

    # Valley
    P2PIDX =  np.concatenate([pks_PPG[:-1][..., np.newaxis], pks_PPG[1:][..., np.newaxis] ], axis=-1)
    val_PPG =  np.unique([np.argmin(PPG[i[0]:i[1]]) +i[0]  for i in P2PIDX])

    valley_dist_PPG = get_difference_list(val_PPG) # valley 간격 리스트
    std_valley_dist_PPG = np.std(valley_dist_PPG) # valley 간격의 std
    std_valley_PPG = np.std(PPG[val_PPG]) # valley 값의 std
    num_valley_PPG = len(val_PPG) # valley 갯수

    # valley 값들이 고르지 않거나, valley 간격이 고르지 않거나, valley 갯수가 길이에 비해 많거나 적을 경우 => bad signal
    if (std_valley_PPG > 0.085) or (std_valley_dist_PPG > 19) or (num_valley_PPG < int(sig_length/(fs*1.5))) or (num_valley_PPG > int(sig_length/(fs*0.5))):
        return 0

    # 첫번째 valley가 너무 늦게 검출되거나, 마지막 valley가 너무 이르게 검출되면 => bad signal
    if (val_PPG[0] > 1.5*np.mean(valley_dist_PPG)) or (sig_length - val_PPG[-1] > 1.5*np.mean(valley_dist_PPG)):
        return 0

    # 추가 feature
    max_min_differ_val = np.max(PPG[val_PPG]) - np.min(PPG[val_PPG]) # 최대 valley와 최소 valley 차이, feature 4
    differ_val = get_difference_list(PPG[val_PPG])
    max_differ_val = max(np.abs(differ_val)) # valley간 차분값의 최댓값, feature 5

    if (max_min_differ_val > 0.19) or (max_differ_val > 0.15):
        return 0

    if np.max(ppg[val_PPG]) >= 40:
        return 0

    # valley 고주파가 심한경우
    PPG_r = 1-PPG
    try:
        wd_p, _ = hp.process(PPG_r, sample_rate=100)
    except Exception:
        return 0

    pks_PPG_r = wd_p['peaklist']
    pks_PPG_r = np.array(pks_PPG_r)
    peak_dist_PPG_r = get_difference_list(pks_PPG_r)
    if np.std(peak_dist_PPG_r)>=30:
        return 0

    GradSignal = np.gradient(PPG)
    NegGradIDX = np.where(GradSignal<0)[0]
    NegGradIDX = np.delete(NegGradIDX,np.where(NegGradIDX==0)) # idx 0의 경우 제외
    gradient_convert = NegGradIDX[GradSignal[NegGradIDX-1]>=0]

    if len(gradient_convert)>=300:
        return 0

    return 1

In [11]:
def check_ppg_abnormal(ppg, sig_length=3500, fs=100):
    if (np.max(ppg) >= 100) or (np.min(ppg) <= 0):
        return {'result': 1, 'reason': '0~100 범위 초과'}

    PPG = minmax_scale(ppg)
    # Transpose the Signals if they are in column format, necessary for peak detection
    if PPG.shape[0] > 1:
        PPG = PPG.T

    try:
        wd_p, _ = hp.process(PPG, sample_rate=fs) # peak 검출
    except Exception:
        return {'result': 1, 'reason': 'heartpy 사용중 error'}

    # Peak
    pks_PPG = wd_p['peaklist'] # peak의 위치 리스트
    pks_PPG = np.array(pks_PPG)

    # Valley
    P2PIDX =  np.concatenate([pks_PPG[:-1][..., np.newaxis], pks_PPG[1:][..., np.newaxis] ], axis=-1)
    val_PPG =  np.unique([np.argmin(PPG[i[0]:i[1]]) +i[0]  for i in P2PIDX])

    if pks_PPG[-1] == (sig_length-1):
        pks_PPG = np.delete(pks_PPG,-1)

    peak_dist_PPG = get_difference_list(pks_PPG) # peak 간격 리스트
    std_peak_dist_PPG = np.std(peak_dist_PPG) # peak 간격의 std, feature 1
    std_peaks_PPG = np.std(PPG[pks_PPG]) # peak 값의 std, feature 2
    num_peaks_PPG = len(pks_PPG) # peak 갯수, feature 3


    # peak 값들이 고르지 않거나, peak 간격이 고르지 않거나, peak 갯수가 길이에 비해 많거나 적을 경우 => bad signal
    if (std_peaks_PPG > 0.1):
        return {'result': 1, 'reason': 'peak들의 std가 0.1 초과'}

    if (std_peak_dist_PPG > 10):
        return {'result': 1, 'reason': 'peak간격의 std가 10 초과'}

    if (num_peaks_PPG < int(sig_length/(fs*1.5))) or (num_peaks_PPG > int(sig_length/(fs*0.5))):
        return {'result': 1, 'reason': '박동이 너무 빠르거나 느림(peak)'}

    # 첫번째 peak가 너무 늦게 검출되거나, 마지막 peak가 너무 이르게 검출되면 => bad signal
    if (pks_PPG[0] > 2*np.mean(peak_dist_PPG)) or (sig_length - pks_PPG[-1] > 2*np.mean(peak_dist_PPG)):
        return {'result': 1, 'reason': '첫번째 peak가 너무 늦게 검출되거나, 마지막 peak가 너무 이르게 검출됌'}

    # 추가 feature
    max_min_differ_pks = np.max(PPG[pks_PPG]) - np.min(PPG[pks_PPG]) # 최대 peak와 최소 peak 차이, feature 4
    differ_pks = get_difference_list(PPG[pks_PPG])
    max_differ_pks = max(np.abs(differ_pks)) # peak간 차분값의 최댓값, feature 5

    if (max_min_differ_pks > 0.3):
        return {'result': 1, 'reason': '최대 peak와 최소 peak 차이가 0.3 초과'}

    if (max_differ_pks > 0.18):
        return {'result': 1, 'reason': 'peak간 차분값의 최댓값이 0.18 초과'}

    if np.min(ppg[pks_PPG]) <= 40:
        return {'result': 1, 'reason': 'peak의 최소값이 40 이하'}


    # valley
    valley_dist_PPG = get_difference_list(val_PPG) # valley 간격 리스트
    std_valley_dist_PPG = np.std(valley_dist_PPG) # valley 간격의 std
    std_valley_PPG = np.std(PPG[val_PPG]) # valley 값의 std
    num_valley_PPG = len(val_PPG) # valley 갯수

    # valley 값들이 고르지 않거나, valley 간격이 고르지 않거나, valley 갯수가 길이에 비해 많거나 적을 경우 => bad signal
    if (std_valley_PPG > 0.1):
        return {'result': 1, 'reason': 'valley들의 std가 0.1 초과'}

    if (std_valley_dist_PPG > 25):
        return {'result': 1, 'reason': 'valley간격의 std가 20 초과'}

    if (num_valley_PPG < int(sig_length/(fs*1.5))) or (num_valley_PPG > int(sig_length/(fs*0.5))):
        return {'result': 1, 'reason': '박동이 너무 빠르거나 느림(valley)'}

    # 추가 feature
    max_min_differ_val = np.max(PPG[val_PPG]) - np.min(PPG[val_PPG]) # 최대 valley와 최소 valley 차이, feature 4
    differ_val = get_difference_list(PPG[val_PPG])
    max_differ_val = max(np.abs(differ_val)) # valley간 차분값의 최댓값, feature 5

    if (max_min_differ_val > 0.3):
        return {'result': 1, 'reason': '최대 valley와 최소 valley 차이가 0.3 초과'}

    if (max_differ_val > 0.18):
        return {'result': 1, 'reason': 'valley간 차분값의 최댓값이 0.18 초과'}

    if np.max(ppg[val_PPG]) >= 40:
        return {'result': 1, 'reason': 'valley의 최대값이 40 이하'}

    # valley 고주파가 심한경우
    PPG_r = 1-PPG
    try:
        wd_p, _ = hp.process(PPG_r, sample_rate=100)
    except Exception:
        return {'result': 1, 'reason': 'heartpy 사용중 error'}

    pks_PPG_r = wd_p['peaklist']
    pks_PPG_r = np.array(pks_PPG_r)
    peak_dist_PPG_r = get_difference_list(pks_PPG_r)
    if np.std(peak_dist_PPG_r)>=30:
        return {'result': 1, 'reason': 'valley 고주파'}

    GradSignal = np.gradient(PPG)
    NegGradIDX = np.where(GradSignal<0)[0]
    NegGradIDX = np.delete(NegGradIDX,np.where(NegGradIDX==0)) # idx 0의 경우 제외
    gradient_convert = NegGradIDX[GradSignal[NegGradIDX-1]>=0]

    if len(gradient_convert)>=400:
        return {'result': 1, 'reason': 'gradient 부호 변화가 400 이상(고주파)'}

    return {'result': 0, 'reason': '정상'}

In [14]:
print(f'check {len(DataSet)} files')

TestSet = []
Reasons = []

for idx, data in enumerate(DataSet[:50]):   # range(len(rawdata_list)):
    print(f'checking {idx}th file')
    normal_ppg_list = list()

    L = data.shape[0]-3500

    for i in range(0, ((L//2500)-1)):
        Sample25 = data[i*2500 : (i+1)*2500]
        Sample35 = data[(i+1)*2500 : ((i+1)*2500 + 3500)]
        result = check_ppg_abnormal(Sample35)
        if check_ppg_normal(Sample25) and result['result']:
            TestSet.append(np.concatenate([Sample25,Sample35]))
            Reasons.append(result['reason'])

print('complete')
print(f'testset 개수: {len(TestSet)}')

check 2823 files
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 error'}
{'result': 1, 'reason': 'heartpy 사용중 e

In [ ]:
np.save(path+ParaFolder+'/TestSet', TestSet)
np.save(path+ParaFolder+'/Reasons', Reasons)

In [ ]:
TestDataSet = np.load(path+ParaFolder+'/TestSet.npy',allow_pickle = True)
Reasons = np.load(path+ParaFolder+'/Reasons.npy',allow_pickle = True)

In [ ]:
DataID = 200
plt.figure(figsize=(20,18))

plt.subplot(511)
plt.plot(TestDataSet[DataID])

plt.subplot(512)
plt.plot(TestDataSet[DataID+1])

plt.subplot(513)
plt.plot(TestDataSet[DataID+2])

plt.subplot(514)
plt.plot(TestDataSet[DataID+3])

plt.subplot(515)
plt.plot(TestDataSet[DataID+4])

plt.show()